In [1]:
import cv2
import torch
from onnx2pytorch import ConvertModel
from keras.models import load_model
import onnx

In [2]:

model1 = load_model('/Users/jarvis/pymycod/Deepfakes/MesoNet-DeepFakeDetection/trained_models/model1_18epochs_valacc0.9252.hdf5')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2024-02-01 14:03:45.859532: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-01 14:03:45.859958: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
import numpy as np
import keras.utils as image

img_width, img_height = 256,256
img = image.load_img(f'/Users/jarvis/Downloads/im6.jpeg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
ans = model1.predict(img)
if ans[0] ==0:
    print("the image is fake afff")
else:
    print("the image is realllll boii")

1/1 [==============================] - 0s 198ms/step
the image is realllll boii


2024-02-01 14:03:47.045296: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-02-01 14:03:47.118034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [11]:

onnx_model = onnx.load('/Users/jarvis/pymycod/Deepfakes/multimodal_deepfake_detection/checkpoints/efficientnet.onnx')
# pytorch_model = ConvertModel(onnx_model)

DecodeError: Wrong wire type in tag.

In [ ]:

def load_img_modality_model(args):
    '''Loads image modality model.'''
    rgb_encoder = pytorch_model

    ckpt = torch.load('checkpoints/model.pth', map_location = torch.device('cpu'))
    rgb_encoder.load_state_dict(ckpt['rgb_encoder'], strict = True)
    rgb_encoder.eval()
    return rgb_encoder
img_model = load_img_modality_model(args)

def preprocess_img(face):
    face = face / 255
    face = cv2.resize(face, (256, 256))
    # face = face.transpose(2, 0, 1) #(W, H, C) -> (C, W, H)
    face_pt = torch.unsqueeze(torch.Tensor(face), dim = 0) 
    return face_pt
def preprocess_video(input_video, n_frames = 3):
    v_cap = cv2.VideoCapture(input_video)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Pick 'n_frames' evenly spaced frames to sample
    if n_frames is None:
        sample = np.arange(0, v_len)
    else:
        sample = np.linspace(0, v_len - 1, n_frames).astype(int)

    #Loop through frames.
    frames = []
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
            # Load frame
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = preprocess_img(frame)
            frames.append(frame)
    v_cap.release()
    return frames


def deepfakes_video_predict(input_video):
    '''Perform inference on a video.'''
    video_frames = preprocess_video(input_video)
    real_faces_list = []
    fake_faces_list = []

    for face in video_frames:
        # face = preprocess_img(face)

        img_grads = img_model.forward(face)
        img_grads = img_grads.cpu().detach().numpy()
        img_grads_np = np.squeeze(img_grads)
        real_faces_list.append(img_grads_np[0])
        fake_faces_list.append(img_grads_np[1])

    real_faces_mean = np.mean(real_faces_list)
    fake_faces_mean = np.mean(fake_faces_list)

    if real_faces_mean > 0.5:
        preds = round(real_faces_mean * 100, 3)
        text2 = f"The video is REAL. \nConfidence score is: {preds}%"

    else:
        preds = round(fake_faces_mean * 100, 3)
        text2 = f"The video is FAKE. \nConfidence score is: {preds}%"

    return text2

In [ ]:
deepfakes_video_predict("/Users/jarvis/Documents/Ss/ras_df.mov")

AttributeError: 'Functional' object has no attribute 'forward'

In [10]:
from onnx import checker

checker.check_model("/Users/jarvis/pymycod/Deepfakes/multimodal_deepfake_detection/checkpoints/efficientnet.onnx")


ValidationError: Unable to parse proto from file: /Users/jarvis/pymycod/Deepfakes/multimodal_deepfake_detection/checkpoints/efficientnet.onnx. Please check if it is a valid protobuf file of proto. 